In [ ]:
from pathlib import Path

import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style("darkgrid")

In [ ]:
%pwd

In [ ]:
# We have to move up to the parent directory so that we can import things from utils
%cd ..
%pwd


In [ ]:
import utils

In [ ]:
PROJECT_ROOT = Path(".").resolve()
print(f"project root: {PROJECT_ROOT}")

In [ ]:
redcap_fname = PROJECT_ROOT / "bx" / "redcap_BABIES.csv"
redcap_datadict_fname = PROJECT_ROOT / "bx" / "BABIES_DataDictionary.csv"
assert redcap_fname.exists()
assert redcap_datadict_fname.exists()

In [ ]:
df_redcap = utils.redcap.get_redcap_df(fname=redcap_fname, fname_datadict=redcap_datadict_fname)

In [ ]:
fname_pcirs = PROJECT_ROOT / "bx" / "PCIRS_6mo_FP_wide_scored_cononly_20240329.csv"
df_pcirs = pd.read_csv(fname_pcirs, index_col="ID", usecols=lambda col: col != "Unnamed: 0")
df_pcirs = df_pcirs.reset_index().rename(columns={"ID": "study_id"})
df_pcirs["study_id"] = df_pcirs["study_id"].apply(lambda id_: "sub-" + str(id_))
df_pcirs

In [ ]:
aseg_fname_newborn = PROJECT_ROOT / "csv" / "aseg_newborn_brainvol.table.txt"
aseg_fname_sixmonth = PROJECT_ROOT / "csv" / "aseg_sixmonth_brainvol.table.txt"

df_vol_newborn = utils.io.get_brainvol_df(aseg_fname_newborn)
df_vol_newborn = df_vol_newborn.pivot(columns=["region"]).droplevel(0, axis=1)
df_vol_newborn.columns.name = ""

df_vol_sixmonth = utils.io.get_brainvol_df(aseg_fname_sixmonth)
df_vol_sixmonth = df_vol_sixmonth.pivot(columns=["region"]).droplevel(0, axis=1)
df_vol_sixmonth.columns.name = ""
df_vol_sixmonth

# Newborn

In [ ]:
# Newborn
df_vol_pcirs_newborn = df_vol_newborn.merge(df_pcirs.set_index("study_id"), left_index=True, right_index=True)
df_vol_pcirs_newborn

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(15, 15), constrained_layout=True)

sns.heatmap(df_vol_pcirs_newborn.corr(), annot=False, ax=ax)

In [ ]:
want_cols = [col for col in df_vol_pcirs_newborn.columns if "vol" in col.lower() or col == "senstot"]
df_reduced = df_vol_pcirs_newborn.loc[:, want_cols]
df_reduced.corr()

# Six Month

In [ ]:
df_vol_pcirs_sixmonth = df_pcirs.merge(df_vol_sixmonth, on="study_id").set_index("study_id")
df_vol_pcirs_sixmonth

In [ ]:
fig, ax = plt.subplots(figsize=(15, 15), constrained_layout=True)

sns.heatmap(df_vol_pcirs_sixmonth.corr(), ax=ax)

In [ ]:
want_cols = [col for col in df_vol_pcirs_sixmonth.columns if "vol" in col.lower() or col == "senstot"]
df_reduced = df_vol_pcirs_sixmonth.loc[:, want_cols]
df_reduced

In [ ]:
sns.pairplot(df_reduced)

In [ ]:
df_reduced.corr()

In [ ]:
sns.lmplot(data=df_vol_pcirs_sixmonth, x="CerebralWhiteMatterVol", y="senstot")

In [ ]:
import statsmodels.formula.api as smf

mod = smf.ols(formula="senstot ~ CerebralWhiteMatterVol", data=df_vol_pcirs_sixmonth)
result = mod.fit()
result.summary()

In [ ]:
df_ages = utils.io.get_gestational_age_df().set_index("study_id")
df_ages

In [ ]:
df_pcirs.set_index("study_id").join(df_ages).corr()

In [ ]:
data =df_reduced.merge(df_ages["sixmonth_scan_age_weeks"], left_index=True, right_index=True)
smf.ols(formula="senstot ~ CerebralWhiteMatterVol + sixmonth_scan_age_weeks", data=data).fit().summary()

## Let's examine this relationship for longitudinal data only

In [ ]:
df_longitudinal = df_vol_newborn.merge(df_vol_sixmonth, left_index=True, right_index=True, suffixes=("_newborn", "_sixmonth"))
len(df_longitudinal)

In [ ]:
newborn_cols = [col for col in df_longitudinal.columns if col.endswith("_newborn")]
session_mapping = {newborn_col: newborn_col.replace("_newborn", "_sixmonth") for newborn_col in newborn_cols}

for col in newborn_cols:
    df_longitudinal[f"{col.replace('_newborn', '')}_growth"] = df_longitudinal[session_mapping[col]] - df_longitudinal[col]

In [ ]:
df_longitudinal

In [ ]:
want_cols = [col for col in df_longitudinal.columns if "growth" in col.lower()]
want_cols

In [ ]:
df_pcirs_vol_growth = df_pcirs.set_index("study_id").merge(df_longitudinal[want_cols], left_index=True, right_index=True)
df_pcirs_vol_growth

In [ ]:
corr_matrix = df_pcirs_vol_growth.corr()

In [ ]:
fig, ax = plt.subplots(figsize=(15, 15), constrained_layout=True)

sns.heatmap(df_pcirs_vol_growth.corr(), annot=False, ax=ax)

In [ ]:
df_pcirs_vol_growth.loc[:, ["stimtot"] + want_cols].corr()

# Surface outputs

In [ ]:
aparc_lh_newborn = PROJECT_ROOT / "csv" / "aparc_newborn_lh.csv"
aparc_rh_newborn = PROJECT_ROOT / "csv" / "aparc_newborn_rh.csv"

aparc_lh_sixmonth = PROJECT_ROOT / "csv" / "aparc_sixmonth_lh.csv"
aparc_rh_sixmonth = PROJECT_ROOT / "csv" / "aparc_sixmonth_rh.csv"

df_hemis_newborn = utils.io.get_aparc_all_hemisphere(aparc_lh_newborn, aparc_rh_newborn).set_index("study_id")
df_hemis_sixmonth = utils.io.get_aparc_all_hemisphere(aparc_lh_sixmonth, aparc_rh_sixmonth).set_index("study_id")

In [ ]:
df_hemis_sixmonth

In [ ]:
# df_hemis_sixmonth = df_hemis_sixmonth.set_index("study_id")
cond_1 = df_hemis_sixmonth["metric"] == "SurfArea"
cond_2 = df_hemis_sixmonth["hemisphere"] == "lh"
df_sixmonth_surf_lh = df_hemis_sixmonth.loc[cond_1 & cond_2].drop(columns=["metric", "hemisphere"]).pivot(columns=["StructName"]).droplevel(0, axis=1)
df_sixmonth_surf_lh


In [ ]:
df_sixmonth_surf_lh_pcirs = df_sixmonth_surf_lh.merge(df_pcirs.set_index("study_id"), left_index=True, right_index=True)

In [ ]:
df_sixmonth_surf_lh_pcirs

In [ ]:
fig, ax = plt.subplots(figsize=(20, 20), constrained_layout=True)

corr = df_sixmonth_surf_lh_pcirs.corr()
sns.heatmap(corr, ax=ax)

In [ ]:
dont_want_cols = [
    col for col in df_sixmonth_surf_lh_pcirs.columns
    if "neg" in col.lower()
    or "tot" in col.lower()
    or "pos" in col.lower()
    or "detach" in col.lower()
    or "intrus" in col.lower()
    or "sens" in col.lower()
    or "stim" in col.lower()
    ]
corr.loc[corr.index == "senstot", [col for col in df_sixmonth_surf_lh_pcirs.columns if col not in dont_want_cols]]


In [ ]:
smf.ols(formula="senstot ~ ctx_superiorfrontal", data=df_sixmonth_surf_lh_pcirs.rename(columns={"ctx-superiorfrontal":"ctx_superiorfrontal"})).fit().summary()